In [1]:
import numpy
from matplotlib import pyplot, cm
from math import pi
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [3]:
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

print('dx =',dx)
print('dy =',dx)

dx = 0.025
dy = 0.025


In [4]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

In [5]:
x = numpy.linspace(0,l,nx)
y = numpy.linspace(0,1,ny)

In [10]:
omega = numpy.zeros((nx,ny))
psi = omega.copy()

We know that,

$$ \nabla^2 \omega = 0 $$

and using jacobian iteration, we get

$$ \omega_{i,j} ^{k+1} = \frac{1}{4} (\omega_{i,j-1} ^{k} +\omega_{i,j+1} ^{k} \
    +  \omega_{i-1,j} ^{k} + \omega_{i+1,j} ^{k}) $$
    
Similarly, for

$$ \nabla^2 \psi = -\omega $$

we can discretize it just like the laplace eqn,

$$ \psi_{i,j}^{k+1} = \frac{(\psi_{i+1,j}^{k} +\psi_{i+1,j}^{k}) \Delta x^2 \
                        (\psi_{i,j+1}^{k} +\psi_{i,j-1}^{k}) \Delta x^2 \ 
                        + \omega_{i,j}^{k} \Delta x^2 \Delta y^2}
                        {2(\Delta x^2 + \Delta y^2) }$$

In [42]:
def stokes(omega,psi,l1_target,u):
    
    l1_norm = 1
    iterations = 0
    
    omegad = numpy.empty_like(omega)
    psid = numpy.empty_like(psi)
    
    while l1_norm > l1_target:
    
        omegad = omega.copy()
        psid = psi.copy()
    
        omega[1:-1,1:-1] = (1/4) * (omegad[1:-1,2:] +omegad[1:-1,:-2] \
                                    + omegad[2:,1:-1] + omegad[:-2,1:-1])
        

        psi[1:-1,1:-1] = 1/(2*(dx**2 + dy**2)) * \
                       ((psid[1:-1,2:]+psid[1:-1,:-2])*dy**2 + \
                        (psid[2:,1:-1] + psid[:-2,1:-1])*dx**2 + \
                        omegad[1:-1,1:-1]*dx**2*dy**2)
        
        #Top surface BC
        omega[-1,:] = -1/(2*dy**2) * (8*psi[-2,:] - psi[-3,:]) - 3*u /dy 
        
        omega[0,:] = -1/(2*dy**2) * (8*psi[1,:] - psi[2,:])                       
        omega[:,-1] = -1/(2*dx**2) * (8*psi[:,-2] - psi[:,-3])                            
        omega[:,0] = -1/(2*dx**2) * (8*psi[:,1] - psi[:,2])
                                    
                                    
    return 